In [29]:
import pdfplumber
import pandas as pd
import re


In [31]:
# Initialisation d'une liste pour stocker le texte extrait
texte_extrait = []

# Ouvrir le fichier PDF
with pdfplumber.open('description-des-bases-de-donnees-onisr-annees-2005-a-2020.pdf') as pdf:

    # Parcourir chaque page du PDF
    for page in pdf.pages:
        texte = page.extract_text()
        texte_extrait.append(texte)

texte_complet = "\n".join(texte_extrait)
print(texte_complet)



Observatoire national interministériel
de la sécurité routière
Le 10 novembre 2021
Description des bases de données annuelles
des accidents corporels de la circulation routière
Années de 2005 à 2020
Pour chaque accident corporel (soit un accident survenu sur une voie ouverte à la circulation publique,
impliquant au moins un véhicule et ayant fait au moins une victime ayant nécessité des soins), des
saisies d’information décrivant l’accident sont effectuées par l’unité des forces de l’ordre (police,
gendarmerie, etc.) qui est intervenue sur le lieu de l’accident. Ces saisies sont rassemblées dans
une fiche intitulée bulletin d’analyse des accidents corporels. L’ensemble de ces fiches constitue le
fichier national des accidents corporels de la circulation dit « Fichier BAAC » administré par
l’Observatoire national interministériel de la sécurité routière "ONISR".
Les bases de données, extraites du fichier BAAC, répertorient l'intégralité des accidents corporels de
la circulation, interve

Nom et descriptif "Carac"

In [49]:
caracteristique_re = re.compile("(?<=La rubrique CARACTERISTIQUES\n)(.+\n)+(?=La rubrique LIEUX)")

In [51]:
carac = caracteristique_re.search(texte_complet).group(0)

In [84]:
carac

"Num_Acc\nNuméro d'identifiant de l’accident.\njour mois\nJour de l'accident.\nMois de l'accident.\nan\nAnnée de l'accident.\nhrmn\nHeure et minutes de l'accident.\nlum\nLumière : conditions d’éclairage dans lesquelles l'accident s'est produit :\n1 – Plein jour\n2 – Crépuscule ou aube\n3 – Nuit sans éclairage public\n4 – Nuit avec éclairage public non allumé\n5 – Nuit avec éclairage public allumé\ndep\nDépartement : Code INSEE (Institut National de la Statistique et des Etudes Economiques) du\ndépartement (2A Corse-du-Sud – 2B Haute-Corse).\ncom\nCommune : Le numéro de commune est un code donné par l‘INSEE. Le code est composé du code\nINSEE du département suivi par 3 chiffres.\nagg\nLocalisation :\n1 – Hors agglomération\n2 – En agglomération\nint\nIntersection :\n1 – Hors intersection\n2 – Intersection en X\n3 – Intersection en T\n4 – Intersection en Y\n5 – Intersection à plus de 4 branches\n6 – Giratoire\n7 – Place\n8 – Passage à niveau\n9 – Autre intersection\n4\natm\nConditions at

In [86]:

# Définir un motif regex pour extraire les données
pattern = re.compile(r"(?P<var>\w+)\n(?P<desc>(?:.*?\n)+?)(?=\w+\n|$)", re.MULTILINE | re.DOTALL)

# Extraire les données en utilisant le motif
matches = pattern.finditer(carac)

# Préparer les listes pour le DataFrame
var_names = []
col_names = []
descriptions = []

for match in matches:
    var_name = match.group('var')
    description = match.group('desc').strip()

    # Filtrer les lignes non pertinentes comme celle contenant "4" pour la page du PDF
    if var_name.isdigit() and len(var_name) == 1:
        continue  # Exclure les lignes contenant uniquement un chiffre

    # Traitement de la description pour extraire des variables qui pourraient être sur plusieurs lignes
    desc_lines = description.split('\n')
    clean_desc = []
    additional_info = []

    for line in desc_lines:
        line = line.strip()
        if line and not re.match(r'^\d+', line):
            # Suppose que les lignes qui ne commencent pas par un chiffre sont des descriptions
            clean_desc.append(line)
        else:
            # Ajouter toute ligne de description supplémentaire à une autre liste si nécessaire
            additional_info.append(line)
    
    # Combiner les lignes de description proprement
    full_description = ' '.join(clean_desc)
    
    var_names.append(var_name)
    col_names.append(var_name)
    descriptions.append(full_description)

# Créer le DataFrame
df = pd.DataFrame({
    'Variable': var_names,
    'Colonne': col_names,
    'Description': descriptions
})

print(df)


   Variable  Colonne                                        Description
0   Num_Acc  Num_Acc  Numéro d'identifiant de l’accident. jour mois ...
1        an       an                               Année de l'accident.
2      hrmn     hrmn                    Heure et minutes de l'accident.
3       lum      lum  Lumière : conditions d’éclairage dans lesquell...
4       dep      dep  Département : Code INSEE (Institut National de...
5       com      com  Commune : Le numéro de commune est un code don...
6       agg      agg                                     Localisation :
7       int      int                                     Intersection :
8       col      col             Type de collision : -1 – Non renseigné
9       adr      adr  Adresse postale : variable renseignée pour les...
10      lat      lat                                           Latitude
11     Long     Long                                          Longitude
